## Datawrangling the CDC BRFSS (Centers for Disease Control and Prevention, Behavioral Risk Factor Surveillance System) Dataset

#### Section 1: Load, merge CDC datasets

### Table of Contents
1. [Objective](#1.-Objective)
2. [The Dataset](#2.-The-Dataset)
3. [Import Libraries](#3.-Import-Libraries)
4. [Load Data](#4.-Load-Data)


### 1. Objective and Problem Statement

The objective of this data wrangling step is to download datasets from the CDC repository, extract and transform the downloaded files into a compatible format, and organize two years' worth of data into two distinct files. This requires merging the datasets to ensure consistency. Additionally, the available features will be reviewed to identify and remove any unrelated, inconsistent, or error-prone columns.

### 2. The Dataset

The [CDC dataset](https://www.cdc.gov/brfss/annual_data/annual_2023.html) consists of two XPT files for the years [2022](https://www.cdc.gov/brfss/annual_data/2022/files/LLCP2022XPT.zip) and [2023, 2024](https://www.cdc.gov/brfss/annual_data/2023/files/LLCP2023XPT.zip), exported from SAS V9.4 in XPT transport format. The files contain 326 variables for 2022 and 350 variables for 2023, representing combined landline and cellular telephone survey data from states that conducted multiple questionnaire versions and used optional modules.

The two files will be merged into a single dataset using an intersection merge to retain only the common features. Columns deemed unrelated or unimportant to the mental health objective will be analyzed and removed during initial processing.

Column descriptions are provided in the accompanying codebook files (located in the 'data' directory):

- USCODE22_LLCP_102523.HTML ([CDC Link](https://www.cdc.gov/brfss/annual_data/2022/zip/codebook22_llcp-v2-508.zip))
- USCODE23_LLCP_091024.HTML ([CDC Link](https://www.cdc.gov/brfss/annual_data/2023/zip/codebook23_llcp-v2-508.zip))

The raw XPT data files (also located in the 'data' directory):

- LLCP2022.XPT (1.16GB) ([CDC Link](https://www.cdc.gov/brfss/annual_data/2022/files/LLCP2022XPT.zip))
- LLCP2023.XPT (1.2GB) ([CDC Link](https://www.cdc.gov/brfss/annual_data/2023/files/LLCP2023XPT.zip))

### 3. Import Libraries

In [1]:
# Import Libraries
import pandas as pd
import numpy as np

# Set default dataframe behaviors
pd.set_option('display.max_columns', None)

# Autoload local libraries, if any
%reload_ext autoreload
%autoreload 2

### 4. Load Data

In [2]:
import requests
import zipfile
import os

# Download and unpack CDC data files
# For the purpose of reducing repository space, download the zip files when first staging the project.

cdc_2022_zip_url = 'https://www.cdc.gov/brfss/annual_data/2022/files/LLCP2022XPT.zip'
cdc_2023_zip_url = 'https://www.cdc.gov/brfss/annual_data/2023/files/LLCP2023XPT.zip'
cdc_2022_zip_file = 'data/LLCP2022.zip'
cdc_2023_zip_file = 'data/LLCP2023.zip'

# Download 2022 file
if not os.path.exists(cdc_2022_zip_file):
  response = requests.get(cdc_2022_zip_url)
  if response.status_code == 200:
    with open(cdc_2022_zip_file, 'wb') as file:
      file.write(response.content)
  else:
    print(f'Download failed for {cdc_2022_zip_url}')

# Download 2023 file
if not os.path.exists(cdc_2023_zip_file):
  response = requests.get(cdc_2023_zip_url)
  if response.status_code == 200:
    with open(cdc_2023_zip_file, 'wb') as file:
      file.write(response.content)
  else:
    print(f'Download failed for {cdc_2023_zip_url}')

# Extract zip files, then delete
if os.path.exists(cdc_2022_zip_file):
  with zipfile.ZipFile(cdc_2022_zip_file, 'r') as zip_ref:
    zip_ref.extractall('data/')
  os.remove(cdc_2022_zip_file)
if os.path.exists(cdc_2023_zip_file):
  with zipfile.ZipFile(cdc_2023_zip_file, 'r') as zip_ref:
    zip_ref.extractall('data/')
  os.remove(cdc_2023_zip_file)


In [3]:
# Load and convert 2022, 2023 XPT files to CSV

_xpt_2022 = 'data/LLCP2022.XPT ' # Extract zip file leaves space in the file name.
_xpt_2023 = 'data/LLCP2023.XPT '

if os.path.exists(_xpt_2022):
  try:
    _df = pd.read_sas(_xpt_2022, format='xport')
    _df.to_csv('./data/LLCP2022.csv')
  except Exception as e:
    print('Error reading file: ', e)
  os.remove(_xpt_2022)

if os.path.exists(_xpt_2023):
  try:
    _df = pd.read_sas(_xpt_2023, format='xport')
    _df.to_csv('./data/LLCP2023.csv')
  except Exception as e:
    print('Error reading file: ', e)
  os.remove(_xpt_2023)


In [4]:
# Load 2023 CSV Data into a dataframe
_2023_df = pd.read_csv('./data/LLCP2023.csv', index_col=0)

In [5]:
# Load 2022 CSV Data into a dataframe
_2022_df = pd.read_csv('./data/LLCP2022.csv', index_col=0)

In [6]:
_2023_df.head()

,_STATE,FMONTH,IDATE,IMONTH,IDAY,IYEAR,DISPCODE,SEQNO,_PSU,CTELENM1,PVTRESD1,COLGHOUS,STATERE1,CELPHON1,LADULT1,NUMADULT,RESPSLC1,LANDSEX2,LNDSXBRT,SAFETIME,CTELNUM1,CELLFON5,CADULT1,CELLSEX2,CELSXBRT,PVTRESD3,CCLGHOUS,CSTATE1,LANDLINE,HHADULT,SEXVAR,GENHLTH,PHYSHLTH,MENTHLTH,POORHLTH,PRIMINS1,PERSDOC3,MEDCOST1,CHECKUP1,EXERANY2,EXRACT12,EXEROFT1,EXERHMM1,EXRACT22,EXEROFT2,EXERHMM2,STRENGTH,BPHIGH6,BPMEDS1,CHOLCHK3,TOLDHI3,CHOLMED3,CVDINFR4,CVDCRHD4,CVDSTRK3,ASTHMA3,ASTHNOW,CHCSCNC1,CHCOCNC1,CHCCOPD3,ADDEPEV3,CHCKDNY2,HAVARTH4,DIABETE4,DIABAGE4,MARITAL,EDUCA,RENTHOM1,NUMHHOL4,NUMPHON4,CPDEMO1C,VETERAN3,EMPLOY1,CHILDREN,INCOME3,PREGNANT,WEIGHT2,HEIGHT3,DEAF,BLIND,DECIDE,DIFFWALK,DIFFDRES,DIFFALON,FALL12MN,FALLINJ5,SMOKE100,SMOKDAY2,USENOW3,ECIGNOW2,ALCDAY4,AVEDRNK3,DRNK3GE5,MAXDRNKS,FLUSHOT7,FLSHTMY3,PNEUVAC4,SHINGLE2,HIVTST7,HIVTSTD3,SEATBELT,DRNKDRI2,COVIDPO1,COVIDSM1,COVIDACT,PDIABTS1,PREDIAB2,DIABTYPE,INSULIN1,CHKHEMO3,EYEEXAM1,DIABEYE1,DIABEDU1,FEETSORE,ARTHEXER,ARTHEDU,LMTJOIN3,ARTHDIS2,JOINPAI2,LCSFIRST,LCSLAST,LCSNUMCG,LCSCTSC1,LCSSCNCR,LCSCTWHN,HADMAM,HOWLONG,CERVSCRN,CRVCLCNC,CRVCLPAP,CRVCLHPV,HADHYST2,PSATEST1,PSATIME1,PCPSARS2,PSASUGS1,PCSTALK2,HADSIGM4,COLNSIGM,COLNTES1,SIGMTES1,LASTSIG4,COLNCNCR,VIRCOLO1,VCLNTES2,SMALSTOL,STOLTEST,STOOLDN2,BLDSTFIT,SDNATES1,CNCRDIFF,CNCRAGE,CNCRTYP2,CSRVTRT3,CSRVDOC1,CSRVSUM,CSRVRTRN,CSRVINST,CSRVINSR,CSRVDEIN,CSRVCLIN,CSRVPAIN,CSRVCTL2,INDORTAN,NUMBURN3,SUNPRTCT,WKDAYOUT,WKENDOUT,CIMEMLO1,CDWORRY,CDDISCU1,CDHOUS1,CDSOCIA1,CAREGIV1,CRGVREL4,CRGVLNG1,CRGVHRS1,CRGVPRB3,CRGVALZD,CRGVPER1,CRGVHOU1,CRGVEXPT,LASTSMK2,STOPSMK2,MENTCIGS,MENTECIG,HEATTBCO,FIREARM5,GUNLOAD,LOADULK2,HASYMP1,HASYMP2,HASYMP3,HASYMP4,HASYMP5,HASYMP6,STRSYMP1,STRSYMP2,STRSYMP3,STRSYMP4,STRSYMP5,STRSYMP6,FIRSTAID,ASPIRIN,BIRTHSEX,SOMALE,SOFEMALE,TRNSGNDR,MARIJAN1,MARJSMOK,MARJEAT,MARJVAPE,MARJDAB,MARJOTHR,USEMRJN4,ACEDEPRS,ACEDRINK,ACEDRUGS,ACEPRISN,ACEDIVRC,ACEPUNCH,ACEHURT1,ACESWEAR,ACETOUCH,ACETTHEM,ACEHVSEX,ACEADSAF,ACEADNED,IMFVPLA4,HPVADVC4,HPVADSHT,TETANUS1,COVIDVA1,COVACGE1,COVIDNU2,LSATISFY,EMTSUPRT,SDLONELY,SDHEMPLY,FOODSTMP,SDHFOOD1,SDHBILLS,SDHUTILS,SDHTRNSP,SDHSTRE1,RRCLASS3,RRCOGNT2,RRTREAT,RRATWRK2,RRHCARE4,RRPHYSM2,RCSGEND1,RCSXBRTH,RCSRLTN2,CASTHDX2,CASTHNO2,QSTVER,QSTLANG,_METSTAT,_URBSTAT,MSCODE,_STSTR,_STRWT,_RAWRAKE,_WT2RAKE,_IMPRACE,_CHISPNC,_CRACE1,CAGEG,_CLLCPWT,_DUALUSE,_DUALCOR,_LLCPWT2,_LLCPWT,_RFHLTH,_PHYS14D,_MENT14D,_HLTHPL1,_HCVU653,_TOTINDA,METVL12_,METVL22_,MAXVO21_,FC601_,ACTIN13_,ACTIN23_,PADUR1_,PADUR2_,PAFREQ1_,PAFREQ2_,_MINAC12,_MINAC22,STRFREQ_,PAMISS3_,PAMIN13_,PAMIN23_,PA3MIN_,PAVIG13_,PAVIG23_,PA3VIGM_,_PACAT3,_PAINDX3,_PA150R4,_PA300R4,_PA30023,_PASTRNG,_PAREC3,_PASTAE3,_RFHYPE6,_CHOLCH3,_RFCHOL3,_MICHD,_LTASTH1,_CASTHM1,_ASTHMS1,_DRDXAR2,_MRACE1,_HISPANC,_RACE,_RACEG21,_RACEGR3,_RACEPRV,_SEX,_AGEG5YR,_AGE65YR,_AGE80,_AGE_G,HTIN4,HTM4,WTKG3,_BMI5,_BMI5CAT,_RFBMI5,_CHLDCNT,_EDUCAG,_INCOMG1,_SMOKER3,_RFSMOK3,_CURECI2,DRNKANY6,DROCDY4_,_RFBING6,_DRNKWK2,_RFDRHV8,_FLSHOT7,_PNEUMO3,_AIDTST4,_RFSEAT2,_RFSEAT3,_DRNKDRV
0,1.0,1.0,b'03012023',b'03',b'01',b'2023',1100.0,b'2023000001',2.023000e+09,1.0,1.0,NaN,1.0,2.0,1.0,2.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,88.0,88.0,NaN,3.0,1.0,2.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,888.0,1.0,1.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,NaN,2.0,2.0,2.0,2.0,2.0,2.0,1.0,57.0,1.0,5.0,1.0,2.0,NaN,1.0,2.0,7.0,88.0,99.0,NaN,172.0,503.0,2.0,2.0,2.0,1.0,2.0,2.0,88.0,NaN,2.0,NaN,3.0,1.0,888.0,NaN,NaN,NaN,2.0,NaN,2.0,2.0,2.0,NaN,1.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [7]:
_2022_df.IYEAR.value_counts()

IYEAR
b'2022'    419473
b'2023'     25659
Name: count, dtype: int64

The dataset contains some data points from 2024, indicating that it spans three years of survey data.

### 5. Initial Data Cleaning, Merging Datasets

#### Removing Unrelated and Redundant Features

Based on the data codebook, some features were identified as unrelated to the mental health model's objectives, such as entries related to tanning or aspirin use, and will be removed.

Features specific to landline or cellphone-based surveys will be combined into common features.
For example:
- HHADULT and LADULT1 refer to cellphone-sourced data for respondents aged 18 and above.
- LADULT refers to the same age range from landline-sourced survey data.

These features will be merged to avoid redundancy and ensure consistency across survey sources.

Additionally, survey entries marked as incomplete, indicated by the value 1200 in the DISPCODE column, will also be excluded from the dataset.

In [8]:
# Leave out incomplete interviews.

# Drop columns flagged as incomplete interview (1200)
# 1100: Completed Interview
_2023_df = _2023_df[_2023_df['DISPCODE'] == 1100]
_2022_df = _2022_df[_2022_df['DISPCODE'] == 1100]

# Then drop DISPSCODES - not relevant 
_2023_df = _2023_df.drop(columns='DISPCODE')
_2022_df = _2022_df.drop(columns='DISPCODE')

#### List of Excluded Columns. 
See the Data Codebook for Reference.

In [9]:
# List of columns to be removed

drop_columns = [
  'FALL12MN', 'FALLINJ5', 'FLUSHOT7', 'FLSHTMY3', 'PNEUVAC4', 'SHINGLE2', 'HIVTST7',
  'CRGVEXPT',
  'FMONTH', 'CTELENM1', 'PVTRESD1', 'COLGHOUS', 'RESPSLC1', 'SAFETIME', 'CTELNUM1',
  'CELLFON5', 'PVTRESD3', 'CCLGHOUS', 'RENTHOM1', 'NUMHHOL4', 'NUMPHON4', 'CPDEMO1C',
  'FALL12MN', 'FALLINJ5', 'FLUSHOT7', 'FLSHTMY3', 'PNEUVAC4', 'SHINGLE2', 'HIVTST7',
  'SEATBELT', 'DRNKDRI2', 'COVIDPO1', 'COVIDSM1', 'COVIDACT', 'PDIABTS1', 'PREDIAB2',
  'DIABTYPE', 'INSULIN1', 'CHKHEMO3', 'EYEEXAM1', 'DIABEYE1', 'DIABEDU1', 'FEETSORE',
  'ARTHEXER', 'ARTHEDU', 'LMTJOIN3', 'ARTHDIS2', 'JOINPAI2', 'LCSFIRST', 'LCSLAST',
  'LCSCTSC1', 'LCSSCNCR', 'LCSCTWHN', 'HADMAM', 'HOWLONG', 'CERVSCRN', 'CRVCLCNC', 'CRVCLPAP',
  'CRVCLHPV', 'HADHYST2', 'PSATEST1', 'PSATIME1', 'PCPSARS2', 'PSASUGS1', 'PCSTALK2',
  'HADSIGM4', 'COLNSIGM', 'COLNTES1', 'SIGMTES1', 'LASTSIG4', 'COLNCNCR', 'VIRCOLO1',
  'VCLNTES2', 'SMALSTOL', 'STOLTEST', 'STOOLDN2', 'BLDSTFIT', 'SDNATES1', 'CNCRDIFF',
  'CNCRAGE', 'CNCRTYP2', 'CSRVTRT3', 'CSRVDOC1', 'CSRVSUM', 'CSRVRTRN', 'CSRVINST',
  'CSRVINSR', 'CSRVDEIN', 'CSRVCLIN', 'CSRVPAIN', 'CSRVCTL2',
  # Indoor Tanning, Sun Exposure
  'INDORTAN', 'NUMBURN3', 'SUNPRTCT', 
  # Caregiver
  'CRGVREL4', 'CRGVLNG1', 'CRGVHRS1', 'CRGVPRB3', 'CRGVALZD', 'CRGVPER1', 'CRGVHOU1',
  'CRGVEXPT',
  # Stopped Smoking
  'LASTSMK2', 'STOPSMK2', 'MENTCIGS', 'MENTECIG', 'HEATTBCO', 
  # Firearms safety - keep ('FIREARM5')
  'GUNLOAD', 'LOADULK2', 
  # Stroke and Heart Attack
  'HASYMP1', 'HASYMP2', 'HASYMP3', 'HASYMP4', 'HASYMP5', 'HASYMP6', 'STRSYMP1', 'STRSYMP2',
  'STRSYMP3', 'STRSYMP4', 'STRSYMP5', 'STRSYMP6', 'FIRSTAID', 
  # CVD Prevention?
  'ASPIRIN', 
  # Marijuana
  'MARJSMOK', 'MARJEAT', 'MARJVAPE', 'MARJDAB', 'MARJOTHR', 
  # Childhood Trauma
  'ACEDRINK', 'ACEDRUGS', 'ACEPRISN', 'ACEDIVRC', 'ACEPUNCH', 'ACEHURT1', 'ACESWEAR', 'ACETOUCH',
  'ACETTHEM', 'ACEHVSEX', 'ACEADSAF', 'ACEADNED', 
  # Vaccine
  'IMFVPLA4', 'HPVADVC4', 'HPVADSHT', 'TETANUS1', 'COVIDVA1', 'COVACGE1', 'COVIDNU2',  
  # Race features
  'RRCOGNT2', 'RRTREAT', 'RRATWRK2', 'RRHCARE4', 'RRPHYSM2', 
  # Child selection?
  'RCSGEND1', 'RCSXBRTH', 'RCSRLTN2', 'CASTHDX2', 'CASTHNO2', 
  # Questionnaire version
  'QSTVER', 
 # Data stats
 'MSCODE', '_STSTR', '_STRWT', '_RAWRAKE', '_WT2RAKE', '_IMPRACE', '_CHISPNC',
 '_CRACE1', 'CAGEG', '_CLLCPWT', '_DUALUSE', '_DUALCOR', '_LLCPWT2', '_LLCPWT',
 '_RFHLTH', '_PHYS14D', '_HLTHPL1', '_HCVU653', '_TOTINDA', 'METVL12_',
 'METVL22_', 'MAXVO21_', 'FC601_', 'ACTIN13_', 'ACTIN23_', 'PADUR1_', 'PADUR2_',
 'PAFREQ1_', 'PAFREQ2_', '_MINAC12', '_MINAC22', 'STRFREQ_', 'PAMISS3_', 'PAMIN13_', 
 'PAMIN23_', 'PA3MIN_', 'PAVIG13_', 'PAVIG23_', 'PA3VIGM_', '_PACAT3', '_PAINDX3', 
 '_PA150R4', '_PA300R4', '_PA30023', '_PASTRNG', '_PAREC3', '_PASTAE3', '_RFHYPE6',
 '_CHOLCH3', '_RFCHOL3', '_MICHD', '_LTASTH1', '_CASTHM1', '_ASTHMS1', '_DRDXAR2', 
 '_MRACE1', '_HISPANC', '_RACE', '_RACEG21', '_RACEGR3', '_RACEPRV', '_SEX', '_AGEG5YR',
 '_AGE65YR', '_AGE80', '_AGE_G', 'HTIN4', 'HTM4', 'WTKG3', '_BMI5', '_BMI5CAT', '_RFBMI5',
 '_CHLDCNT', '_EDUCAG', '_INCOMG1', '_SMOKER3', '_RFSMOK3', '_CURECI2', 'DRNKANY6',
 'DROCDY4_', '_RFBING6', '_DRNKWK2', '_RFDRHV8', '_FLSHOT7', '_PNEUMO3', '_AIDTST4',
 '_RFSEAT2', '_RFSEAT3', '_DRNKDRV',
 # Others
  'CELPHON1', 'SEQNO','HEIGHT3', 'WEIGHT2', 'IDATE', '_PSU', 'NUMADULT', 'LANDLINE',
  'HHADULT', 'ECIGNOW2', 'EXRACT12', 
  'EXRACT22', 'TOLDHI3', 'ASTHMA3', 'DIABAGE4',
  'HIVTSTD3', 'LCSNUMCG', 'CAREGIV1',
  'SOMALE', 'SOFEMALE', 'USEMRJN4', 'CHILDREN',
  'MAXDRNKS', 'SMOKE100', 'EXERANY2', 'EXERHMM1', 'AVEDRNK3','DRNK3GE5',
  'EXERHMM2', 'EXEROFT2', 'STRENGTH', 'WKDAYOUT', 'WKENDOUT',
  'PRIMINS1', 'PERSDOC3', 'DEAF', 'BLIND', 'IDAY', 'IMONTH', 'IYEAR'
]


In [10]:
# Drop columns from 2023

_2023_df = _2023_df.drop(columns=drop_columns)

Get intersections of 2023 and 2022 data.

The 2022 data contains common columns with 2023, apply row-merge along the common columns (ignore the unique columns of 2022), and keep the unique columns of 2023.

In [11]:
# Get column names from 2023
set_2023 = set(_2023_df.columns)
# Get column names from 2022 
set_2022 = set(_2022_df.columns)
# Apply intersect and get the common columns
common = set_2022.intersection(set_2023)
# Get columns found only in 2023
uniq_2023 = list(set_2023 - common)
# Get columns found only in 2022
uniq_2022 = list(set_2022 - common)

print(f'Common Cols ({len(common)}): ', list(common))
print(f'2023 Unique Cols ({len(uniq_2023)}): ', uniq_2023)
print(f'2022 Unique Cols ({len(uniq_2022)}): ', uniq_2022)

Common Cols (55):  ['EDUCA', 'EMTSUPRT', 'SDHEMPLY', 'DIFFDRES', 'CHCCOPD3', 'VETERAN3', 'POORHLTH', 'SMOKDAY2', 'CHECKUP1', 'SDHUTILS', 'EMPLOY1', 'MENTHLTH', 'ADDEPEV3', '_STATE', '_METSTAT', 'FIREARM5', 'DIABETE4', '_MENT14D', 'SDHSTRE1', 'LADULT1', 'LSATISFY', 'HAVARTH4', 'CVDINFR4', 'TRNSGNDR', 'SEXVAR', 'PHYSHLTH', 'ACEDEPRS', 'INCOME3', 'MARITAL', 'BIRTHSEX', 'FOODSTMP', 'SDHTRNSP', 'CSTATE1', 'CVDCRHD4', 'CVDSTRK3', 'DIFFALON', 'DIFFWALK', 'CHCOCNC1', 'SDHBILLS', 'RRCLASS3', 'ALCDAY4', 'CADULT1', 'CHCSCNC1', 'ASTHNOW', 'MARIJAN1', 'CHCKDNY2', 'PREGNANT', 'MEDCOST1', 'GENHLTH', 'QSTLANG', 'STATERE1', 'SDHFOOD1', 'USENOW3', 'DECIDE', '_URBSTAT']
2023 Unique Cols (15):  ['CDDISCU1', 'BPMEDS1', 'CDWORRY', 'LNDSXBRT', 'EXEROFT1', 'CIMEMLO1', 'CHOLMED3', 'BPHIGH6', 'SDLONELY', 'CDHOUS1', 'CELLSEX2', 'LANDSEX2', 'CHOLCHK3', 'CELSXBRT', 'CDSOCIA1']
2022 Unique Cols (272):  ['MARJEAT', 'MARJVAPE', 'NUMADULT', 'HADMAM', 'CRGVLNG1', 'DIABEDU1', '_BMI5', '_FLSHOT7', 'ACETOUCH', '_PACKYRS',

In [12]:
# Drop uniq_2022 columns.
_2022_df = _2022_df.drop(columns=uniq_2022)

# Now both datasets have the same number of features.

# Merge datasets
merged_df = pd.concat([_2023_df, _2022_df], ignore_index=True)

In [13]:
# Visual inspection
merged_df.head()

,_STATE,STATERE1,LADULT1,LANDSEX2,LNDSXBRT,CADULT1,CELLSEX2,CELSXBRT,CSTATE1,SEXVAR,GENHLTH,PHYSHLTH,MENTHLTH,POORHLTH,MEDCOST1,CHECKUP1,EXEROFT1,BPHIGH6,BPMEDS1,CHOLCHK3,CHOLMED3,CVDINFR4,CVDCRHD4,CVDSTRK3,ASTHNOW,CHCSCNC1,CHCOCNC1,CHCCOPD3,ADDEPEV3,CHCKDNY2,HAVARTH4,DIABETE4,MARITAL,EDUCA,VETERAN3,EMPLOY1,INCOME3,PREGNANT,DECIDE,DIFFWALK,DIFFDRES,DIFFALON,SMOKDAY2,USENOW3,ALCDAY4,CIMEMLO1,CDWORRY,CDDISCU1,CDHOUS1,CDSOCIA1,FIREARM5,BIRTHSEX,TRNSGNDR,MARIJAN1,ACEDEPRS,LSATISFY,EMTSUPRT,SDLONELY,SDHEMPLY,FOODSTMP,SDHFOOD1,SDHBILLS,SDHUTILS,SDHTRNSP,SDHSTRE1,RRCLASS3,QSTLANG,_METSTAT,_URBSTAT,_MENT14D
0,1.0,1.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,2.0,2.0,88.0,88.0,NaN,2.0,2.0,NaN,1.0,1.0,3.0,2.0,2.0,2.0,2.0,NaN,2.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,5.0,2.0,7.0,99.0,NaN,2.0,1.0,2.0,2.0,NaN,3.0,888.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,2.0,1.0,5.0,2.0,2.0,5.0,2.0,2.0,2.0,5.0,NaN,1.0,1.0,1.0,1.0
1,1.0,1.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,2.0,2.0,88.0,88.0,NaN,2.0,2.0,106.0,1.0,2.0,2.0,1.0,2.0,2.0,2.0,NaN,2.0,2.0,2.0,1.0,2.0,1.0,3.0,2.0,5.0,2.0,7.0,99.0,NaN,2.0,2.0,2.0,2.0,NaN,3.0,888.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,1.0,2.0,3.0,2.0,2.0,5.0,2.0,2.0,2.0,5.0,NaN,1.0,1.0,1.0,1.0
2,1.0,1.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,2.0,4.0,6.0,2.0,1.0,1.0,1.0,205.0,1.0,1.0,2.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,3.0,3.0,4.0,2.0,7.0,2.0,NaN,1.0,1.0,2.0,2.0,3.0,3.0,888.0,1.0,2.0,1.0,2.0,2.0,NaN,NaN,4.0,NaN,NaN,2.0,4.0,3.0,2.0,2.0,5.0,1.0,1.0,1.0,3.0,NaN,1.0,1.0,1.0,2.0
3,1.0,1.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,2.0,2.0,2.0,88.0,88.0,2.0,3.0,103.0,3.0,NaN,3.0,2.0,2.0,2.0,2.0,NaN,1.0,1.0,2.0,1.0,2.0,1.0,3.0,1.0,5.0,2.0,7.0,99.0,NaN,2.0,1.0,2.0,2.0,NaN,3.0,888.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,1.0,1.0,3.0,2.0,2.0,5.0,2.0,2.0,2.0,5.0,NaN,1.0,2.0,1.0,1.0
4,1.0,1.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,2.0,4.0,88.0,88.0,NaN,2.0,1.0,102.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,NaN,2.0,1.0,2.0,1.0,2.0,1.0,1.0,3.0,5.0,2.0,8.0,7.0,NaN,1.0,1.0,2.0,1.0,NaN,2.0,202.0,1.0,1.0,2.0,2.0,2.0,NaN,NaN,4.0,NaN,NaN,2.0,2.0,2.0,2.0,2.0,4.0,2.0,2.0,2.0,2.0,NaN,1.0,1.0,1.0,1.0


In [14]:
merged_df.isnull().sum()

_STATE           0
STATERE1    550765
LADULT1     550765
LANDSEX2    648573
LNDSXBRT    709395
             ...  
RRCLASS3    418191
QSTLANG          0
_METSTAT     16324
_URBSTAT     16324
_MENT14D         0
Length: 70, dtype: int64

In [16]:
# Save merged data
merged_df.to_csv('./data/llcp_2022_2023.csv')
os.remove('data/LLCP2022.csv')
os.remove('data/LLCP2023.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'data/LLCP2022.csv'